In [1]:
import pandas as pd
from mysql import connector

In [2]:
# Step 1: Connect to SQL server
connection = connector.connect(
    host="localhost",
    user="root",
    port=3306,
    password="123456789",
    connection_timeout=600 # Timeout after 600 seconds
)

mycursor = connection.cursor()


In [3]:
# Step 2: Create database and select it
mycursor.execute("CREATE DATABASE IF NOT EXISTS BusDetails")
mycursor.execute("USE BusDetails")

In [4]:
# Step 3: Define the table schema
table_name = "bus_routes"
column_definitions = """
id INT AUTO_INCREMENT PRIMARY KEY,
route_name TEXT,
route_link TEXT,
busname TEXT,
bustype TEXT,
departing_time TIME,
duration TEXT,
reaching_time TIME,
star_rating FLOAT,
price DECIMAL(10, 2),
seats_available INT,
state TEXT
"""

# Create table
mycursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({column_definitions})")
print(f"Table {table_name} created successfully.")

Table bus_routes created successfully.


In [5]:
# Step 4: Read the CSV file
file_path = r"C:\Users\ashwi\GUVI_Projects\Redbus_Data_Scrapping\bus_details.csv"
try:
    data = pd.read_csv(file_path)
    data = data.dropna()  # Drop rows with null values
    print("CSV file loaded successfully.")
except Exception as e:
    print(f"Error reading the CSV file: {e}")
    connection.close()
    exit()
#data['Seat_Availability'] = data['Seat_Availability'].str.extract(r'(\d+)')

CSV file loaded successfully.


In [6]:
# Step 5: Insert data into the table
try:
    for i in range(len(data)):
        # Convert row to tuple and handle data type conversion for floats
        row = tuple(
            [float(val) if isinstance(val, float) else val for val in data.iloc[i]]
        )
        placeholders = ", ".join(["%s"] * len(row))  # Generate placeholders
        query = f"INSERT INTO {table_name} (route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available, state) VALUES ({placeholders})"
        mycursor.execute(query, row)  # Parameterized query to insert data

    connection.commit()
    print("Data inserted successfully.")
except Exception as e:
    print(f"Error inserting data into the table: {e}")
    connection.close()
    exit()


Data inserted successfully.


In [7]:
# Step 6: Retrieve and display data
try:
    print("\nTables in the database:")
    mycursor.execute("SHOW TABLES")
    for db in mycursor:
        print(db)

    print("\nData in the table:")
    mycursor.execute(f"SELECT * FROM {table_name}")
    for record in mycursor:
        print(record)
except Exception as e:
    print(f"Error retrieving data: {e}")
finally:
    # Step 7: Close the connection
    connection.close()
    print("Connection closed.")


Tables in the database:
('bus_routes',)

Data in the table:
(1, 'Tezpur to Guwahati', 'https://www.redbus.in/bus-tickets/tezpur-to-guwahati', 'Assam State Transport Corporation (ASTC) - 157959', 'Bharat Benz A/C Seater (2+2)', datetime.timedelta(seconds=50400), '04h 30m', datetime.timedelta(seconds=66600), 4.1, Decimal('298.00'), 13, 'Assam')
(2, 'Tezpur to Guwahati', 'https://www.redbus.in/bus-tickets/tezpur-to-guwahati', 'Mahalaxmi Travels', 'A/C Seater (2+2)', datetime.timedelta(seconds=35100), '03h 45m', datetime.timedelta(seconds=48600), 3.8, Decimal('300.00'), 32, 'Assam')
(3, 'Tezpur to Guwahati', 'https://www.redbus.in/bus-tickets/tezpur-to-guwahati', 'Aziba Travels', 'NON A/C Seater (2+1)', datetime.timedelta(seconds=35100), '04h 25m', datetime.timedelta(seconds=51000), 4.0, Decimal('347.00'), 5, 'Assam')
(4, 'Guwahati to Tezpur', 'https://www.redbus.in/bus-tickets/guwahati-to-tezpur', 'Assam State Transport Corporation (ASTC) - 154910', 'Bharat Benz A/C Seater (2+2)', dateti